In [1]:
import tensorflow as tf
import math
import os
import time
import itertools
import numpy as np
from data_helpers import RWBatchGenerator

In [2]:
tf.flags.DEFINE_string('data_dir', 'data/stackexchange/datascience/', 'directory of dataset')
tf.flags.DEFINE_integer("checkpoint_every", 5000, "Save model after this many steps (default: 5000)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

In [3]:
FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
CHECKPOINT_EVERY=5000
DATA_DIR=data/stackexchange/datascience/
NUM_CHECKPOINTS=5



In [4]:
data_dir = FLAGS.data_dir

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 3       # How many words to consider left and right.
num_skips = 4  # How many times to reuse an input to generate a label.

walks = RWBatchGenerator.read_walks("{}/random_walks.txt".format(data_dir))

vocabulary_size = len(set(itertools.chain(*walks)))

generator = RWBatchGenerator(walks, batch_size, num_skips, skip_window)

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.



In [5]:
graph = tf.Graph()

with graph.as_default():
    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        with tf.name_scope('embedding'):
            embeddings = tf.Variable(
                tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        with tf.name_scope('nce'):
            # Construct the variables for the NCE loss
            nce_weights = tf.Variable(
                tf.truncated_normal([vocabulary_size, embedding_size],
                                    stddev=1.0 / math.sqrt(embedding_size)))
            nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(
            tf.nn.nce_loss(weights=nce_weights,
                           biases=nce_biases,
                           labels=train_labels,
                           inputs=embed,
                           num_sampled=num_sampled,
                           num_classes=vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

In [ ]:
timestamp = str(int(time.time()))
out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "deepwalk"))
#                                       "deepwalk-{}".format(timestamp)))
if tf.gfile.Exists(out_dir):
    tf.gfile.DeleteRecursively(out_dir)
tf.gfile.MakeDirs(out_dir)

print("Writing to {}\n".format(out_dir))

# summary config
loss_summary = tf.summary.scalar("loss", loss)
train_summary_op = tf.summary.merge([loss_summary])
train_summary_dir = os.path.join(out_dir, "summaries", "train")

# checkpoint config
checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
checkpoint_prefix = os.path.join(checkpoint_dir, "model")
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

Writing to /home/cloud-user/code/network_embedding/runs/deepwalk



In [ ]:
# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
    train_summary_writer = tf.summary.FileWriter(train_summary_dir, session.graph)
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)    
    
    
    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generator.next_batch()
        
        feed_dict = {train_inputs: batch_inputs,
                     train_labels: np.expand_dims(np.array(batch_labels), -1)}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val, summaries = session.run([optimizer, loss, train_summary_op], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = valid_examples[i]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = nearest[k]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)

        if step % FLAGS.checkpoint_every == 0 and step > 0:
            path = saver.save(session, checkpoint_prefix, global_step=step)
            
            print("Saved model checkpoint to {}\n".format(path))                
            
        train_summary_writer.add_summary(summaries, step)
        
    final_embeddings = normalized_embeddings.eval()
    

    from tensorflow.contrib.tensorboard.plugins import projector

    embedding_var = tf.Variable(final_embeddings,  name='node_embedding')
    session.run(embedding_var.initializer)
    
    # Format: tensorflow/tensorboard/plugins/projector/projector_config.proto
    config = projector.ProjectorConfig()

    # You can add multiple embeddings. Here we add only one.
    embedding = config.embeddings.add()
    embedding.tensor_name = embedding_var.name

    # Link this tensor to its metadata file (e.g. labels).
    # embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

    # Use the same LOG_DIR where you stored your checkpoint.
    summary_writer = tf.summary.FileWriter(checkpoint_dir)

    # The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
    # read this file during startup.
    projector.visualize_embeddings(summary_writer, config)
    
    saver = tf.train.Saver([embedding_var])
    saver.save(session, os.path.join(checkpoint_dir, 'model2.ckpt'), 1)    
    print('embedding saved')


Initialized
Average loss at step  0 :  194.297409058
Nearest to 22: 2192, 4343, 2517, 58, 2077, 2945, 1535, 1091,
Nearest to 16: 3579, 4155, 5194, 5380, 5173, 259, 339, 5599,
Nearest to 56: 1085, 1336, 2581, 4262, 3166, 1190, 2204, 4733,
Nearest to 47: 2134, 113, 2294, 5805, 1751, 3174, 4094, 5505,
Nearest to 44: 5275, 1026, 445, 5950, 4258, 866, 1007, 5610,
Nearest to 64: 689, 105, 4471, 901, 5374, 4663, 3997, 2325,
Nearest to 78: 5058, 3980, 3376, 211, 2855, 1279, 1126, 4730,
Nearest to 31: 5057, 1625, 4603, 4318, 3356, 4895, 4439, 4662,
Nearest to 84: 6081, 3046, 3996, 59, 1816, 4060, 2046, 3202,
Nearest to 4: 4121, 995, 3574, 1022, 629, 2104, 3366, 3470,
Nearest to 76: 4193, 1681, 4184, 2012, 2468, 4738, 2287, 725,
Nearest to 62: 4716, 1928, 1519, 2207, 1791, 3488, 6031, 1645,
Nearest to 91: 895, 3224, 3312, 3218, 2581, 3958, 1329, 3525,
Nearest to 57: 3961, 2637, 1044, 2351, 669, 677, 4010, 313,
Nearest to 55: 5369, 3571, 1909, 1798, 1246, 702, 4115, 2998,
Nearest to 49: 5492, 273